In [1]:
import cv2 as cv
import numpy as np

In [2]:
confThreshold = 0.25 #leaving out all boxes below this probability
nmsThreshold = 0.40
inpWidth = 416 # input width for pictures
inpHeight = 416 # input height for pictures

In [3]:
#load names of classes and turn into a list
classesFile = "darknet/coco.names.txt"
classes = None

with open(classesFile, 'r') as f:
    classes = f.read().strip('\n').split('\n')

FileNotFoundError: [Errno 2] No such file or directory: 'darknet/coco.names.txt'

In [4]:
#model configuration
modelConf = 'darknet/cfg/yolov3.cfg'
modelWeights = 'darknet/yolov3.weights'

In [5]:
def postprocess(frame, outs):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]
    
    classIDs = []
    confidences = []
    boxes = []
    
    #create a list for results
    # results = []
    # see results = []
    
    #return the tensor, going through each vector in the tensor
    for out in outs:
        for detection in out:
            #print(len(detection)) return vector of 85% values
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID] #take class ID, return the percentage value for the object detected, here the confidence threshold comes into play
            
            if confidence > confThreshold:
                centerX = int(detection[0] * frameWidth) #perc. value
                centerY = int(detection[1] * frameHeight) #perc. value
                
                width = int(detection[2] + frameWidth)
                height = int(detection[2] + frameWidth)
                
                left = int(centerX - width/2)
                top = int(centerY - height/2)
                
                classIDs.append(classID) #the object the model was confident enough to predict
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])
                
    indices = cv.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    
    for i in indices:
        i = i[0]
        box = boxes[i] #iterating over the boxes
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[2]
        #print(left, top, width, height)
        
        drawPred(classIDs[i], confidences[i], left, top, left + width, top + height)
        
def drawPred(classID, conf, left, top, right, bottom):
    cv.rectangle(frame, (left, top), (right, bottom), (255,178,50), 3)
    
    label = "%.2f" % conf
    
    if classes:
        assert(classID < len(classes))
        label = "%s:%s" % (classes[classID], label)
        
    cv.putText(frame, label,(left, top), cv.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2) #go to frame, write the label (defined above), and put it in the left corner
        
        
        
        

In [6]:
def getOutputsNames(net):
    layerNames = net.getLayerNames()
    return [layerNames[i[0]-1] for i in net.getUnconnectedOutLayers()]

In [7]:
net = cv.dnn.readNetFromDarknet(modelConf, modelWeights)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

In [ ]:
# Process inputs
winName = "DL OD with OpenCV"
cv.namedWindow(winName, cv.WINDOW_NORMAL)
cv.resizeWindow(winName, 750, 750)

#specifying what video input the model will take, '0' is the webcam
cap = cv.VideoCapture('Youtube Videos/Mosh Pit & Wall of Death montage.mp4')


while cv.waitKey(1) < 0:
    hasFrame, frame = cap.read()
    #take the frame, then rescale each pixel by 1/255 (colors in array of the image) turn the colors in the image into percentages, and then rescale the image, and [0,0,0], 1 is default and we don't want to crop
    blob = cv.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)
    
    net.setInput(blob)
    outs = net.forward(getOutputsNames(net)) #get all the unconnected layers in the network (aka output layers)
    
    postprocess(frame, outs)
    cv.imshow(winName, frame)
cv.destroyAllWindows()